<h1>Classification using SciKit Learn</h1>

<h3> SVM - Simple example</h3>

In [11]:
# http://scikit-learn.org/stable/modules/svm.html

from sklearn import svm

In [20]:
X = [[0,0], [1,1], [2,2], [3,3]]
Y = [0, 1, 2, 3]
clf = svm.SVC()
clf.fit(X, Y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [21]:
dec = clf.decision_function([[1,1]])
dec.shape[1] # 4 classes: 4*3/2 = 6

6

In [24]:
print clf.predict([1.0, 2.0])

[2]


<h3>SVM test with features </h3>

In [26]:
import numpy as np
import pandas as pd
pd.set_option('html',False)
from os import path

In [60]:
P03_FEATURES = path.relpath('data/FEATURES_P03.csv')
LABEL = path.relpath('data/P03_DATA_WINDOW/P03_LABEL_L.csv')

df_features = pd.read_csv(P03_FEATURES, sep='\,')
df_label = pd.read_csv(LABEL, header=None, sep='\ ')

# Get equal lengths of dataframes
df_features = df_features[:len(df_label)]

In [61]:
from sklearn import cross_validation
from sklearn import svm

<h3> Divide data set into training and testing </h3>

In [84]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(df_features,df_label, test_size=0.2, random_state=0)

In [85]:
print X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


(4968, 60) (1242, 60) (4968, 1) (1242, 1)


<h4> Train and test different SVM </h4>

In [93]:
clf = svm.SVC(kernel='linear', C=1.0).fit(X_train, Y_train[0].values)
print 'SVM - svc', clf.score(X_test, Y_test[0].values)

SVM - svc 0.799516908213


ValueError: Unsupported set of arguments: loss='l1' and penalty='squared_hinge'are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True